In [3]:
!pip install selenium

In [19]:
import z3
from z3 import Solver, Int, Real, If, ToReal, ToInt, sat

def tax_solver_113_internal(
    basic_living_exp_per_person: int = 210000,
    savings_investment_deduction_limit: int = 270000,
    disability_deduction_per_person: int = 218000,
    education_deduction_per_student: int = 25000,
    long_term_care_deduction_per_person: int = 120000,
    rent_deduction_limit: int = 180000,  # 房屋租金特別扣除上限

    # 免稅額 (依附件)
    personal_exemption_under70: int = 97000,
    personal_exemption_over70: int  = 145500,

    standard_deduction_single: int  = 131000,
    standard_deduction_married: int = 262000,
    salary_special_deduction_max: int = 218000,

    # 速算公式級距 (同原程式)
    bracket1_upper: int = 590000,
    bracket2_upper: int = 1330000,
    bracket3_upper: int = 2660000,
    bracket4_upper: int = 4980000,

    bracket1_rate: float = 0.05,
    bracket2_rate: float = 0.12,
    bracket3_rate: float = 0.20,
    bracket4_rate: float = 0.30,
    bracket5_rate: float = 0.40,

    bracket1_sub: int = 0,
    bracket2_sub: int = 41300,
    bracket3_sub: int = 147700,
    bracket4_sub: int = 413700,
    bracket5_sub: int = 911700,

    # --- 以下為「使用者申報情況」 ---
    is_married: bool = False,
    salary_self: int = 0,
    salary_spouse: int = 0,
    salary_dep: int = 0,
    interest_income: int = 0,
    stock_dividend: int = 0,
    house_transaction_gain: int = 0,
    other_income: int = 0,

    # 這裡的 cnt_under_70 與 cnt_over_70 就是「整個申報戶」各年齡段的人數
    cnt_under_70: int = 0,
    cnt_over_70: int = 0,

    use_itemized: bool = False,
    itemized_deduction: int = 0,

    property_loss_deduction: int = 0,
    disability_count: int = 0,
    education_count: int = 0,
    education_fee: int = 0,
    preschool_count: int = 0,  # 幼兒人數
    long_term_care_count: int = 0,
    rent_deduction: int = 0  # 房屋租金支出特別扣除(使用者輸入)
):
    s = Solver()

    # === 1. 定義 Z3 變數 ===
    salary_self_z   = Int('salary_self_z')
    salary_spouse_z = Int('salary_spouse_z')
    salary_dep_z    = Int('salary_dep_z')
    interest_z      = Int('interest_z')
    stock_div_z     = Int('stock_div_z')
    house_gain_z    = Int('house_gain_z')
    other_income_z  = Int('other_income_z')
    itemized_ded_z  = Int('itemized_ded_z')
    prop_loss_ded_z = Int('prop_loss_ded_z')
    rent_ded_z      = Int('rent_ded_z')

    total_income_z  = Int('total_income_z')

    self_salary_special_deduction_z   = Int('self_salary_special_deduction_z')
    spouse_salary_special_deduction_z = Int('spouse_salary_special_deduction_z')
    dep_salary_special_ded_z          = Int('dep_salary_special_ded_z')

    self_salary_after_ded_z   = Int('self_salary_after_ded_z')
    spouse_salary_after_ded_z = Int('spouse_salary_after_ded_z')
    dep_salary_after_ded_z    = Int('dep_salary_after_ded_z')

    total_exemption_z    = Int('total_exemption_z')
    standard_deduction_z = Int('standard_deduction_z')
    chosen_deduction_z   = Int('chosen_deduction_z')

    savings_investment_deduction_z = Int('savings_investment_deduction_z')
    disability_deduction_z         = Int('disability_deduction_z')
    education_deduction_z          = Int('education_deduction_z')

    preschool_count_z    = Int('preschool_count_z')
    preschool_deduction_z= Int('preschool_deduction_z')

    long_term_care_deduction_z     = Int('long_term_care_deduction_z')
    rent_deduction_z_lim           = Int('rent_deduction_z_lim')

    total_people_z = Int('total_people_z')
    basic_living_exp_total_z = Int('basic_living_exp_total_z')
    base_deductions_var_z = Int('base_deductions_var_z')
    basic_living_exp_diff_z = Int('basic_living_exp_diff_z')
    total_deduction_z = Int('total_deduction_z')
    net_taxable_income_z = Int('net_taxable_income_z')
    net_taxable_nonneg_z = Int('net_taxable_nonneg_z')

    rate_calc_r = Real('rate_calc_r')
    final_tax_z = Int('final_tax_z')

    # === 2. 建立「輸入值 -> Z3 變數」及範圍檢查 ===
    s.add(salary_self_z   == salary_self)
    s.add(salary_spouse_z == salary_spouse)
    s.add(salary_dep_z    == salary_dep)
    s.add(interest_z      == interest_income)
    s.add(stock_div_z     == stock_dividend)
    s.add(house_gain_z    == house_transaction_gain)
    s.add(other_income_z  == other_income)
    s.add(itemized_ded_z  == itemized_deduction)
    s.add(prop_loss_ded_z == property_loss_deduction)
    s.add(rent_ded_z      == rent_deduction)

    s.add(salary_self_z   >= 0)
    s.add(salary_spouse_z >= 0)
    s.add(salary_dep_z    >= 0)
    s.add(interest_z      >= 0)
    s.add(stock_div_z     >= 0)
    s.add(house_gain_z    >= 0)
    s.add(other_income_z  >= 0)
    s.add(itemized_ded_z  >= 0)
    s.add(prop_loss_ded_z >= 0)
    s.add(rent_ded_z      >= 0, rent_ded_z <= 10000000)

    s.add(self_salary_special_deduction_z == If(salary_self_z >= salary_special_deduction_max, salary_special_deduction_max, salary_self_z))
    s.add(spouse_salary_special_deduction_z == If(salary_spouse_z >= salary_special_deduction_max, salary_special_deduction_max, salary_spouse_z))
    s.add(dep_salary_special_ded_z == If(salary_dep_z >= salary_special_deduction_max, salary_special_deduction_max, salary_dep_z))

    s.add(self_salary_after_ded_z == salary_self_z - self_salary_special_deduction_z)
    s.add(spouse_salary_after_ded_z == salary_spouse_z - spouse_salary_special_deduction_z)
    s.add(dep_salary_after_ded_z == salary_dep_z - dep_salary_special_ded_z)

    s.add(self_salary_after_ded_z >= 0)
    s.add(spouse_salary_after_ded_z >= 0)
    s.add(dep_salary_after_ded_z >= 0)

    s.add(total_income_z == (self_salary_after_ded_z + spouse_salary_after_ded_z + dep_salary_after_ded_z + interest_z + stock_div_z + house_gain_z + other_income_z))

    s.add(total_exemption_z == (cnt_under_70 * personal_exemption_under70 + cnt_over_70 * personal_exemption_over70))
    s.add(standard_deduction_z == (standard_deduction_married if is_married else standard_deduction_single))
    s.add(chosen_deduction_z == If(use_itemized, itemized_ded_z , standard_deduction_z))

    interest_plus_div_z = Int('interest_plus_div_z')
    s.add(interest_plus_div_z == (interest_z + stock_div_z))
    s.add(savings_investment_deduction_z == If(interest_plus_div_z <= savings_investment_deduction_limit, interest_plus_div_z, savings_investment_deduction_limit))
    s.add(disability_deduction_z == disability_count * disability_deduction_per_person)
    s.add(education_deduction_z == If(education_fee <= 0, 0, If(education_fee >= education_count * education_deduction_per_student, education_count * education_deduction_per_student, education_fee)))
    s.add(preschool_count_z == preschool_count)
    s.add(preschool_deduction_z == If(preschool_count_z <= 0, 0, If(preschool_count_z == 1, 150000, 150000 + (preschool_count_z - 1) * 225000)))
    s.add(long_term_care_deduction_z == long_term_care_count * long_term_care_deduction_per_person)
    s.add(rent_deduction_z_lim == If(rent_ded_z >= rent_deduction_limit, rent_deduction_limit, rent_ded_z))
    s.add(total_people_z == cnt_under_70 + cnt_over_70)
    s.add(basic_living_exp_total_z == total_people_z * basic_living_exp_per_person)
    s.add(base_deductions_var_z == (total_exemption_z + chosen_deduction_z + savings_investment_deduction_z + disability_deduction_z + education_deduction_z + preschool_deduction_z + long_term_care_deduction_z + rent_deduction_z_lim))
    s.add(basic_living_exp_diff_z == If(basic_living_exp_total_z > base_deductions_var_z, basic_living_exp_total_z - base_deductions_var_z, 0))
    s.add(total_deduction_z == base_deductions_var_z + prop_loss_ded_z + basic_living_exp_diff_z)
    s.add(net_taxable_income_z == total_income_z - total_deduction_z)
    s.add(net_taxable_nonneg_z == If(net_taxable_income_z < 0, 0, net_taxable_income_z))

    x = ToReal(net_taxable_nonneg_z)
    s.add(rate_calc_r == If(x <= bracket1_upper,
                             (x * bracket1_rate) - bracket1_sub,
                          If(x <= bracket2_upper,
                             (x * bracket2_rate) - bracket2_sub,
                          If(x <= bracket3_upper,
                             (x * bracket3_rate) - bracket3_sub,
                          If(x <= bracket4_upper,
                             (x * bracket4_rate) - bracket4_sub,
                             (x * bracket5_rate) - bracket5_sub)))))
    safe_tax_r = If(rate_calc_r < 0, 0, rate_calc_r)
    s.add(final_tax_z == ToInt(safe_tax_r))

    if s.check() == sat:
        m = s.model()
        return m[final_tax_z].as_long(), m[net_taxable_income_z].as_long()
    else:
        return None, None

def tax_solver_113_final(**kwargs):
    # 第一階段：先使用原始輸入計算
    tax_val, net_income = tax_solver_113_internal(**kwargs)
    # 若淨所得超過 1,330,000 (即進入 20% 以上區間)，則長照及房租扣除不適用
    if net_income is not None and net_income >= 1330000:
        kwargs_modified = kwargs.copy()
        kwargs_modified['long_term_care_count'] = 0
        kwargs_modified['rent_deduction'] = 0
        tax_val2, net_income2 = tax_solver_113_internal(**kwargs_modified)
        return tax_val2
    else:
        return tax_val

# ========== 測試 ==========
if __name__ == "__main__":
    tax = tax_solver_113_final(
        is_married=True,
        salary_self=1795785,
        salary_spouse=589914,
        salary_dep=119016, 
        interest_income=35504,
        stock_dividend=33616,
        house_transaction_gain=674194,
        other_income=152873,
        cnt_under_70=1,
        cnt_over_70=3,
        use_itemized=True,
        itemized_deduction=441883,
        property_loss_deduction=181269,
        disability_count=1,
        education_count=0,
        education_fee=0,
        preschool_count=2,
        long_term_care_count=0,
        rent_deduction=88155
    )
    print("113年度試算  =", tax)


113年度試算  = 71375


### selenium 含113年新的『房屋租金支出特別扣除』

In [22]:
import random
import time
import z3
from z3 import Solver, Int, Real, If, ToReal, ToInt, sat

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

def generate_random_params():
    
    is_married = random.choice([True, False])
    salary_self = random.randint(500000, 2000000)
    salary_spouse = random.randint(500000, 1000000) if is_married else 0
    salary_dep = random.randint(0, 500000)
    interest_income = random.randint(0, 300000)
    stock_dividend = random.randint(0, 200000)
    house_transaction_gain = random.randint(0, 1000000)
    other_income = random.randint(0, 1000000)
    cnt_under_70 = random.randint(0, 2)
    cnt_over_70 = random.randint(0, 4)
    use_itemized = random.choice([True, False])
    itemized_deduction = random.randint(50000, 500000) if use_itemized else 0
    property_loss_deduction = random.randint(0, 500000)
    while True:
        disability_count = random.randint(0, 3)
        education_count = random.randint(0, 3)
        preschool_count = random.randint(0, 3)
        long_term_care_count = random.randint(0, 3)
        if (disability_count + education_count + preschool_count + long_term_care_count) <= (cnt_under_70 + cnt_over_70):
            break
    rent_deduction = random.randint(0, 180000) # 介於 0 ~ 180,000
    education_fee = random.randint(0, 25000 * education_count)

    return {
        "is_married": is_married,
        "salary_self": salary_self,
        "salary_spouse": salary_spouse,
        "salary_dep": salary_dep,
        "interest_income": interest_income,
        "stock_dividend": stock_dividend,
        "house_transaction_gain": house_transaction_gain,
        "other_income": other_income,
        "cnt_under_70": cnt_under_70,
        "cnt_over_70": cnt_over_70,
        "use_itemized": use_itemized,
        "itemized_deduction": itemized_deduction,
        "property_loss_deduction": property_loss_deduction,
        "disability_count": disability_count,
        "education_count": education_count,
        "education_fee": education_fee,
        "preschool_count": preschool_count,
        "long_term_care_count": long_term_care_count,
        "rent_deduction": rent_deduction
    }

def wait_for_stable_tax_value(driver, timeout=15, stable_duration=1):
    end_time = time.time() + timeout
    last_value = None
    stable_start = None
    while time.time() < end_time:
        current_value = driver.find_element(By.ID, "tax").get_property("value").strip()
        # 如果欄位有值
        if current_value != "":
            if last_value is None or current_value != last_value:
                last_value = current_value
                stable_start = time.time()
            else:
                if time.time() - stable_start >= stable_duration:
                    return current_value
        time.sleep(0.5)
    return current_value  # timeout時返回當前值

# 在 get_online_tax_result 中替換等待部分如下：
def get_online_tax_result(params, driver):
    url = "https://www.etax.nat.gov.tw/etwmain/etw158w/15"
    driver.delete_all_cookies()
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "isMarried"))
    )
    
    def fill_field(element, value):
        element.click()
        time.sleep(0.1)
        element.clear()
        time.sleep(0.1)
        for ch in str(value):
            element.send_keys(ch)
            time.sleep(0.05)
        element.send_keys(Keys.TAB)
        time.sleep(0.1)
    
    select_married = Select(driver.find_element(By.ID, "isMarried"))
    select_married.select_by_value("1" if params["is_married"] else "0")
    time.sleep(0.5)
    
    fill_field(driver.find_element(By.ID, "salary"), params["salary_self"])
    
    spouse_field = driver.find_element(By.ID, "spouseSalary")
    if params["is_married"]:
        driver.execute_script("arguments[0].removeAttribute('readonly')", spouse_field)
        fill_field(spouse_field, params["salary_spouse"])
    
    fill_field(driver.find_element(By.ID, "kindred"), params["salary_dep"])
    fill_field(driver.find_element(By.ID, "interest"), params["interest_income"])
    fill_field(driver.find_element(By.ID, "dividend"), params["stock_dividend"])
    fill_field(driver.find_element(By.ID, "tradeIncome"), params["house_transaction_gain"])
    fill_field(driver.find_element(By.ID, "otherIncome"), params["other_income"])
    
    fill_field(driver.find_element(By.ID, "under70"), params["cnt_under_70"])
    fill_field(driver.find_element(By.ID, "upward70"), params["cnt_over_70"])
    
    list_field = driver.find_element(By.ID, "list")
    if params["use_itemized"]:
        fill_field(list_field, params["itemized_deduction"])
    else:
        fill_field(list_field, "0")
    
    fill_field(driver.find_element(By.ID, "wDeduct"), params["property_loss_deduction"])   
    fill_field(driver.find_element(By.ID, "challengedNum"), params["disability_count"])
    fill_field(driver.find_element(By.ID, "eduNum"), params["education_count"])
    fill_field(driver.find_element(By.ID, "eduDeduct"), params["education_fee"])
    fill_field(driver.find_element(By.ID, "afterSchoolNum"), params["preschool_count"])
    fill_field(driver.find_element(By.ID, "longTimeCareNum"), params["long_term_care_count"])
    fill_field(driver.find_element(By.ID, "rentForHousing"), params["rent_deduction"])
    
    try:
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".loading-screen-wrapper"))
        )
    except Exception as e:
        print("Warning: loading screen did not disappear:", e)
    
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@title='試算結果']"))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", button)
    time.sleep(0.5)
    button.click()
    
    # 使用穩定性等待函式
    tax_str = wait_for_stable_tax_value(driver, timeout=15, stable_duration=1)
    tax_str = tax_str.replace(",", "").strip()
    try:
        online_tax = int(tax_str)
    except Exception:
        online_tax = None
    return online_tax

# 測試用的主程式，請根據實際需求修改測試次數
def main():
    total_tests = 100
    mismatches = 0
    
    options = webdriver.ChromeOptions()
    # 如需無頭模式則取消註解下面這行
    # options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    
    for i in range(total_tests):
        params = generate_random_params()
        
        # 這裡假設本地的 SMT solver 計算邏輯在 tax_solver_113 中（未提供具體實作）
        local_tax = tax_solver_113_final(
            is_married=params["is_married"],
            salary_self=params["salary_self"],
            salary_spouse=params["salary_spouse"],
            salary_dep=params["salary_dep"],
            interest_income=params["interest_income"],
            stock_dividend=params["stock_dividend"],
            house_transaction_gain=params["house_transaction_gain"],
            other_income=params["other_income"],
            cnt_under_70=params["cnt_under_70"],
            cnt_over_70=params["cnt_over_70"],
            use_itemized=params["use_itemized"],
            itemized_deduction=params["itemized_deduction"],
            property_loss_deduction=params["property_loss_deduction"],
            disability_count=params["disability_count"],
            education_count=params["education_count"],
            education_fee=params["education_fee"],
            preschool_count=params["preschool_count"],
            long_term_care_count=params["long_term_care_count"],
            rent_deduction=params["rent_deduction"]
        )
        
        online_tax = get_online_tax_result(params, driver)
        
        print(f"Final Local Tax = {local_tax}, Online Tax = {online_tax}")
        if local_tax != online_tax:
            mismatches += 1
            print("  --> Mismatch! Parameters:", params)
    
    print(f"測試結束，共 {total_tests} 筆資料，不一致的筆數：{mismatches}")
    driver.quit()

if __name__ == "__main__":
    main()


Final Local Tax = 248489, Online Tax = 248489
Final Local Tax = 68472, Online Tax = 68472
Final Local Tax = 0, Online Tax = 0
Final Local Tax = 365915, Online Tax = 365915
Final Local Tax = 145029, Online Tax = 145029
Final Local Tax = 78446, Online Tax = 78446
Final Local Tax = 53701, Online Tax = 53701
Final Local Tax = 46637, Online Tax = 46637
Final Local Tax = 328800, Online Tax = 328800
Final Local Tax = 87831, Online Tax = 87831
Final Local Tax = 320548, Online Tax = 320548
Final Local Tax = 414474, Online Tax = 414474
Final Local Tax = 170032, Online Tax = 170032
Final Local Tax = 253363, Online Tax = 253363
Final Local Tax = 0, Online Tax = 0
Final Local Tax = 14087, Online Tax = 14087
Final Local Tax = 163204, Online Tax = 163204
Final Local Tax = 48716, Online Tax = 48716
Final Local Tax = 180698, Online Tax = 180698
Final Local Tax = 0, Online Tax = 0
Final Local Tax = 188923, Online Tax = 188923
Final Local Tax = 116298, Online Tax = 116298
Final Local Tax = 0, Online Tax 